In [2]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Vamos carregar os dados do dataset de diabetes do scikit para fazer uma regressão! Se você apertar **tab** depois do ponto você vai conseguir ver outros datasets que já vem no scikit.

In [3]:
diabetes = datasets.load_diabetes()

In [4]:
X = diabetes.data
y = diabetes.target

Vamos definir uma pipeline que pega nossas features e faz a transformação polinomial deles (da uma olhada na [doc](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)) como primeiro passo. O segundo passo é aplicar a decomposição PCA e, por último, faz uma regressão de Ridge.

In [5]:
pipe = Pipeline(steps=[('polinomial', PolynomialFeatures()),
                       ('pca', PCA()),
                       ('regressao', Ridge())])

Podemos fazer o fit da nossa instância do Pipeline assim como se fosse uma instância de um estimador qualquer.

In [6]:
pipe.fit(X, y)

Pipeline(memory=None,
     steps=[('polinomial', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('regressao', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

Mas o legal de usar o Pipeline é que, como qualquer outro estimador, a gente pode usar ele no cross_validate e no GridSearchCV!

In [7]:
metricas = ['explained_variance',
            'neg_mean_absolute_error',
            'neg_mean_squared_error',
            #'neg_mean_squared_log_error',
            'neg_median_absolute_error',
            'r2']

In [8]:
cross_validate(pipe, X, y, return_train_score=False,
               scoring=metricas)

{'fit_time': array([0.01254892, 0.01198983, 0.00464511]),
 'score_time': array([0.01503205, 0.00954509, 0.00710487]),
 'test_explained_variance': array([0.37125582, 0.41891023, 0.44029588]),
 'test_neg_mean_absolute_error': array([-48.7818243 , -52.70361757, -47.38501551]),
 'test_neg_mean_squared_error': array([-3599.40341608, -3688.61864658, -3188.57772783]),
 'test_neg_median_absolute_error': array([-42.71639399, -48.47971835, -46.09899913]),
 'test_r2': array([0.37045604, 0.41862605, 0.44028555])}

Na hora de fazer o Grid pode ser um pouco confuso saber quais são os parâmetros, já que misturamos muita coisa. O pulo do gato é usar o método ```get_params()```:

In [9]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'polinomial', 'pca', 'regressao', 'polinomial__degree', 'polinomial__include_bias', 'polinomial__interaction_only', 'pca__copy', 'pca__iterated_power', 'pca__n_components', 'pca__random_state', 'pca__svd_solver', 'pca__tol', 'pca__whiten', 'regressao__alpha', 'regressao__copy_X', 'regressao__fit_intercept', 'regressao__max_iter', 'regressao__normalize', 'regressao__random_state', 'regressao__solver', 'regressao__tol'])

In [10]:
param_grid = {'polinomial__degree':[2,3],
              'polinomial__include_bias':[True, False],
              'polinomial__interaction_only':[False],
              'pca__n_components':[None, 10],
              'pca__random_state':[42],
              'pca__whiten':[True, False],
              'regressao__alpha':[1.0, 0.7],
              'regressao__normalize':[False],
              'regressao__random_state':[42],
              'regressao__solver':['svd', 'lsqr', 'saga']
              }

In [11]:
grid = GridSearchCV(pipe, param_grid=param_grid,
                    scoring=metricas, verbose=1,
                    refit='neg_mean_squared_error',
                    return_train_score=False)

In [12]:
grid.fit(X, y)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed:   16.0s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('polinomial', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('regressao', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'polinomial__degree': [2, 3], 'polinomial__include_bias': [True, False], 'polinomial__interaction_only': [False], 'pca__n_components': [None, 10], 'pca__random_state': [42], 'pca__whiten': [True, False], 'regressao__alpha': [1.0, 0.7], 'regressao__normalize': [False], 'regressao__random_state': [42], 'regressao__solver': ['svd', 'lsqr', 'saga']},
       pre_dispatch='2*n_jobs', refit='neg_mean_squared_error',
       return_train_score=False,
       scoring=['explained_var

In [14]:
pd.set_option('max_columns',200)
results = pd.DataFrame(grid.cv_results_)
results.sort_values(by="rank_test_neg_median_absolute_error").head()

,mean_fit_time,mean_score_time,mean_test_explained_variance,mean_test_neg_mean_absolute_error,mean_test_neg_mean_squared_error,mean_test_neg_median_absolute_error,mean_test_r2,param_pca__n_components,param_pca__random_state,param_pca__whiten,param_polinomial__degree,param_polinomial__include_bias,param_polinomial__interaction_only,param_regressao__alpha,param_regressao__normalize,param_regressao__random_state,param_regressao__solver,params,rank_test_explained_variance,rank_test_neg_mean_absolute_error,rank_test_neg_mean_squared_error,rank_test_neg_median_absolute_error,rank_test_r2,split0_test_explained_variance,split0_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error,split0_test_neg_median_absolute_error,split0_test_r2,split1_test_explained_variance,split1_test_neg_mean_absolute_error,split1_test_neg_mean_squared_error,split1_test_neg_median_absolute_error,split1_test_r2,split2_test_explained_variance,split2_test_neg_mean_absolute_error,split2_test_neg_mean_squared_error,split2_test_neg_median_absolute_error,split2_test_r2,std_fit_time,std_score_time,std_test_explained_variance,std_test_neg_mean_absolute_error,std_test_neg_mean_squared_error,std_test_neg_median_absolute_error,std_test_r2
8,0.007757,0.007032,0.402696,-46.824259,-3539.651068,-38.666867,0.401602,None,42,True,2,False,False,1,False,42,saga,"{'pca__n_components': None, 'pca__random_state...",73,25,73,1,73,0.399700,-45.736023,-3445.117814,-38.029483,0.397441,0.413149,-49.440803,-3725.773941,-42.898886,0.412770,0.395260,-45.303356,-3448.704531,-35.076569,0.394624,0.000494,0.000280,0.007598,1.855473,131.393659,3.221730,0.007967
11,0.007219,0.006651,0.402405,-46.828401,-3541.344618,-38.712712,0.401312,None,42,True,2,False,False,0.7,False,42,saga,"{'pca__n_components': None, 'pca__random_state...",74,26,74,2,74,0.399403,-45.737541,-3446.791853,-38.038503,0.397148,0.412962,-49.442152,-3726.962774,-42.994191,0.412583,0.394868,-45.312930,-3450.922443,-35.110028,0.394234,0.000232,0.000708,0.007679,1.853200,131.040070,3.250445,0.008044
65,0.039203,0.039280,0.483306,-44.517191,-3066.085694,-38.728227,0.481970,10,42,True,3,True,False,0.7,False,42,saga,"{'pca__n_components': 10, 'pca__random_state':...",23,21,23,3,23,0.469249,-45.308071,-3050.740846,-42.055209,0.466418,0.483560,-46.477228,-3276.844869,-40.703594,0.483527,0.497205,-41.760893,-2870.775753,-33.403245,0.496072,0.002313,0.002848,0.011421,2.003444,165.946912,3.799273,0.012162
71,0.036259,0.037464,0.483306,-44.517191,-3066.085694,-38.728227,0.481970,10,42,True,3,False,False,0.7,False,42,saga,"{'pca__n_components': 10, 'pca__random_state':...",24,21,24,4,24,0.469249,-45.308071,-3050.740846,-42.055209,0.466418,0.483560,-46.477228,-3276.844869,-40.703594,0.483527,0.497205,-41.760893,-2870.775753,-33.403245,0.496072,0.001364,0.005075,0.011421,2.003444,165.946912,3.799273,0.012162
63,0.042411,0.040020,0.483361,-44.514384,-3065.779924,-38.733575,0.482024,10,42,True,3,True,False,0.7,False,42,svd,"{'pca__n_components': 10, 'pca__random_state':...",15,10,15,5,15,0.469351,-45.301369,-3050.160263,-42.090665,0.466520,0.483546,-46.478208,-3276.929837,-40.716245,0.483514,0.497280,-41.758222,-2870.355929,-33.370978,0.496145,0.001665,0.001353,0.011409,2.004102,166.165207,3.826901,0.012147


Podemos ver que, apesar de nenhum modelo ter ficado particularmente bom (o melhor teve média do erro absoluto de 46, que representa 14% do range de 321 dos valores de y), quando colocamos o PCA com o mesmo número de variáveis "originais" (opção None na coluna param_pca\__n\_components) e o polinomio de grau 2 (coluna param\_polinomial\__degree) os resultados ficam melhores. Se você conseguir um resultado melhor com esse dataset me envia! Até a próxima!

In [22]:
46.824259/ (y.max() - y.min())

0.14586996573208721

In [23]:
(y.max() - y.min())

321.0